# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

In [2]:
# импорт библиотек
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [5]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for k,v in obj.items(): # идем по ключам и значениям словаря и выводим их согласно форматированию
    print(f"'{k:10}' = '{v}'")

'home_page ' = 'https://github.com/pypa/sampleproject'
'keywords  ' = 'sample setuptools development'
'license   ' = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
obj = pd.Series(["Женя группа ПМ21-1", "РОма пм 22-2", "Полина 23-3"]) # формируем случайный набор данных
obj 

0    Женя группа ПМ21-1
1          РОма пм 22-2
2           Полина 23-3
dtype: object

In [4]:
obj.apply(lambda x: re.search(r'\d{2}-\d', x).group()) # прописываем функцию, где рассматриваем символы слева и справа от "-"

0    21-1
1    22-2
2    23-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

In [8]:
string = 'Написать регулярное выражение,которое позволит найти номера групп студентов'
re.split(r'\W', string)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [6]:
recipes = pd.read_csv('recipes_sample.csv') # считываем данные из файла
random5 = recipes.sample(5) # берем случайное значение
max_id = recipes.id.astype(str).str.len().max() + 8 # считаем максимальную длину ID
max_minutes = recipes.minutes.astype(str).str.len().max() + 8 # смотрим макмиальное значение минут
def print_f(id_, minutes): 
    print(
        f'|{id_:^{max_id}}|{minutes:^{max_minutes}}|'
    )

print_f('id', 'minutes') # первая строка для заголовка
print('|'+"-"*(max_id+max_minutes+1)+'|')
for id_, row in random5.iterrows(): # выводим минуты и ID в единой псевдотаблице
    print_f(row.id, row.minutes)

|      id      |   minutes    |
|-----------------------------|
|    42846     |      5       |
|    54004     |     505      |
|    64650     |     100      |
|    302674    |      15      |
|    459186    |     146      |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [7]:
with open('steps_sample.xml') as f: # считываем данные
    steps = BeautifulSoup(f, 'xml')

temp = {
    int(recipe.find('id').next) : [step.next for step in recipe.find_all('step')] # создаем словарь, где ключи это ID, а значения это шаги
    for recipe in steps.find_all('recipe')
}

steps = pd.DataFrame(temp.items(), columns=['id', 'steps']) # переводим наш словарь в датафрейм
result = recipes.merge(steps, how='left', left_on='id', right_on='id') # добавляем в recipes столбец steps 
result.head() # смотрим на результат

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,steps
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,"[in 1 / 4 cup butter , saute carrots , onion ,..."
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,"[mix all the ingredients using a blender, pour..."
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,[combine all ingredients in a large bowl and m...
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,[lay out sandwich rolls on jelly roll pans / c...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,[honey mustard sauce: whisk all the ingredient...


In [8]:
def show_info(name, steps, minutes, author_id):  
    '''
    выводит информацю о рецепте
    количество авторов
    среднее время приготовления
    '''
    steps = '\n'.join([f"{i+1}. {step.capitalize()}" for i,step in enumerate(steps)])
    template = f""""{name.title()}"

    {steps}
    ----------
    Автор: {author_id}
    Среднее время приготовления: {minutes} минут
    """
    return template

target = result[result.id==170895].iloc[0]
print(show_info(target["name"], target.steps, target.minutes, target.contributor_id))

"Leeks And Parsnips  Sauteed Or Creamed"

    1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an 

In [17]:
# утверждение о том, чему должен быть равен вывод функции
assert ( 
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
def task3(string):
    pattern = r'\d+ (?:(?:hours?)|(?:minutes?))\b'
    return re.findall(pattern, string)

target = result[result.id==25082].iloc[0]
[substep for step in target.steps for substep in task3(step)]

['20 minutes',
 '10 minutes',
 '2 hours',
 '10 minutes',
 '20 minutes',
 '30 minutes']

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [10]:
def task4(string):
    if type(string) == float:
        return
    return re.match(r'this(?:\w| )*, ?but', string)

result4 = result[result.description.apply(task4).astype(bool)]
result4.shape[0]

134

In [23]:
for desc in result4.sample(3).description:
    print("-"*20)
    print(desc)

--------------------
this was born one night when i was trying a new recipe, but didn't have all the right ingredients.  it's bits and pieces of 2 recipes plus my own additions.  awesome with fresh mushrooms, but very good with canned as well.  i used to always buy the knorr herb and garlic sauce packages to make this dish, but haven't been able to find them for the last couple of years.  this is very close to the same flavour, if not an exact duplicate, so that's exciting for me! depending on how saucy you like your pasta, this is enough for 2 main course servings and up to 6 side dish servings.
--------------------
this recipe shows for pork, but you can add thinly sliced beef, chicken, or even use shrimp. you can also add just vegetables for a vegetable lo mein. it's very versatile.
--------------------
this is really great to have on hand when you have little time, but want something fresh and hot for breakfast.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [11]:
def task5(steps):
    for i, step in enumerate(steps):
        steps[i] = re.sub(r'(\d+) / (\d+)', r'\1/\2',  step)
        
target = result[result.id==72367].iloc[0]
target.steps

['mix butter , flour , 1 / 3 c',
 'sugar and 1-1 / 4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1 / 4 c',
 'sugar , eggs and 1 / 2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1 / 3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

In [27]:
task5(target.steps)

In [28]:
target.steps

['mix butter , flour , 1/3 c',
 'sugar and 1-1/4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1/4 c',
 'sugar , eggs and 1/2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1/3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
nltk.download()
# only popular!!

In [30]:
result6 = steps.steps.apply(lambda row: [nltk.word_tokenize(step.lower()) for step in row ])
result6 = {word for row in result6 for step in row for word in step}
print(len(result6))
print(result6)

19568
{'egg-lemon-broth', 'pimentos', 'bratwurst', 'meat-', 'vinagar', 'concentric', 'innards', 'pre-sifted', 'his', 'closest', 'mouths', 'mixes', 'dixie', 'church', 'patches', 'jus', 'osem', 'dinner-size', 'thisis', 'cob', 'fancied', '79857', 'conker-dark', 'custard', 'stor', '2-3mins', '15x2-inch', 'youngest', 'scraping', 'matcha', 'eff', 'limited', 'too', 'anginette', 'bouilion', 'unspecified', 'poppers', 'spanish', 'skew', 'burner', 'elk', "cook'til", 'seafood-', 'gases', 'couscous', 'large-sized', 'periels', 'flillet', 'checked', '225c', 'random-sized', 'immersible', 'cones', '10-in', 'medhigh', 'unquote', 'probe', 'sized', 'cherry', 'below', 'pea', '20c', 'remaning', 'stale', '810', 'teaspooon', 'applewood', 'peics', 'neccesary', 're-pack', 'horseradish', 'fillet', 'face-up', 'yokes', 'packetful', 'visable', 'refried', 'non-reactive', 'rocking', 'outline', 'regular-sized', 'uneaten', 'favorable', 'velbekomme', 'lingonberries', 'michoacan', 'recognise', 'spun', '5-10mins', 'alike'

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [32]:
recipes['sent_tokenize'] = recipes.description.fillna("").apply(nltk.sent_tokenize)
five = recipes.sort_values(by="sent_tokenize", key= lambda x: x.str.len(), ascending=False).iloc[:5]
five

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,sent_tokenize
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,[this wonderful icing is used for icing cakes ...
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,[a translucent golden-brown crust allows the g...
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,[this is one of the best soups i've ever made ...
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,[i wrote this because there are an astounding ...
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,[the first time i made this cake i grated a mi...


In [33]:
for id_,row in five.iterrows():
    print(row.description)
    print('*'*50)
    for i, sent in enumerate(row.sent_tokenize):
        print(f'{i})----- {sent}')
    print('*'*50)
    print(len(row.sent_tokenize))
    print('-'*50)
    print('\n\n\n\n')

this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.  it melts!  a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.  

your top notch decorators have a few options we don’t.  t

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [12]:
target = result[result.id==241106].iloc[0]

def task8(sent):
    array = nltk.pos_tag(nltk.word_tokenize(sent))
    temp = [(max([len(i) for i in pair]),*pair) for pair in array]

    for i in range(2,0,-1):
        for item in temp:
            print(f"{item[i]:^{item[0]}}", end=" ")
        print()
        
task8(target["name"])

   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 
